```
Summary by:
- A41316 - Nguyễn Hữu Khoa
- A42718 - Lê Thảo Quyên
```

> **Note:** Many source code is outdated, some parameters, function usage have been changed according to praw

# Data Collection

### Library dependencies

In [1]:
import praw
import sqlite3

### Sqlite storage setup

In [2]:
conn = sqlite3.connect('reddit.db')
c = conn.cursor()

In [3]:
c.execute('''DROP TABLE IF EXISTS topics''')

c.execute('''DROP TABLE IF EXISTS comments''')

c.execute('''CREATE TABLE topics
             (topicTitle text, topicText text, topicID text, topicCategory text)''')

c.execute('''CREATE TABLE comments
             (commentText text, commentID text ,topicTitle text, topicText text, topicID text , topicCategory text)''')

### Praw Reddit connection information

In [18]:
user_agent = "Introducing Data Science Book"
subreddits = ['datascience','gameofthrones']
limit = 100 # number of posts to retrieve

Tạo vào lấy API Reddit tại trang https://old.reddit.com/prefs/apps/ và lấy các thông tin như ảnh dưới đây:
![apiReddit](assets/api.png)

In [5]:
reddit = praw.Reddit(
    client_id="<client_id>", # client_id removed for security
    client_secret="<client_secret>" # client_secret removed for security
    password="<password>", # password removed for security
    user_agent=user_agent,
    username="<username>" # username removed for security
)

### Praw data gathering function

In [13]:
def prawGetData(limit, subredditName):
    topics = reddit.subreddit(subredditName).hot(limit=limit)
    commentInsert = []
    topicInsert = []
    topicNBR = 1
    for topic in topics:
        if (float(topicNBR)/limit)*100 in range(1, 100):
            print('*********** TOPIC:' + str(topic.id) + ' *********COMPLETE: ' + str((float(topicNBR)/limit)*100) + ' % ****')
        topicNBR += 1
        try:
            topicInsert.append((topic.title, topic.selftext, topic.id, subredditName))
        except:
            pass
        try:
            for comment in topic.comments:
                commentInsert.append((comment.body, comment.id, topic.title, topic.selftext, topic.id, subredditName))
        except:
            pass
    print('********************************')
    print('INSERTING DATA INTO SQLITE')
    c.executemany('INSERT INTO topics VALUES (?,?,?,?)', topicInsert)
    print('INSERTED TOPICS')
    c.executemany('INSERT INTO comments VALUES (?,?,?,?,?,?)', commentInsert)
    print('INSERTED COMMENTS')
    conn.commit()


### Executing Data Collection

Khi chạy code phía dưới, 100% bạn sẽ nhận được lỗi 403 
Forbidden: received 403 HTTP response

Lỗi "Forbidden: received 403 HTTP response" xuất hiện khi API của Reddit từ chối truy cập vào một tài nguyên cụ thể, trong trường hợp này là việc truy cập bình luận của một chủ đề hoặc một subreddit.

Lỗi này thường xảy ra khi tài khoản của bạn không có quyền truy cập hoặc bị hạn chế trong việc thao tác với các tài nguyên đó. Điều này có thể xảy ra khi bạn không đăng nhập vào tài khoản Reddit hoặc tài khoản của bạn không có quyền truy cập vào subreddit cụ thể đó. (có thể do subreddit đó đã bị xóa, chuyển thành private, hoặc bạn bị ban khỏi subreddit đó, v.v.)

Để xử lý lỗi này, chúng ta sử dụng cấu trúc try-except để bắt lỗi và thực hiện xử lý phù hợp. Trong trường hợp này, chúng ta muốn tiếp tục thực hiện quá trình lấy dữ liệu và bỏ qua các bình luận không thể truy cập được.

Ta sẽ tạo khối lệnh thêm comment vào một khối try-except bổ sung. Nếu phát sinh lỗi Forbidden, nghĩa là không thể truy cập vào bình luận, chúng ta sẽ bỏ qua bình luận đó và tiếp tục với các bình luận khác. Đồng thời, in ra một thông báo lỗi để ta biết rằng có lỗi xảy ra.

Tuy nhiên, hãy nhớ rằng xử lý lỗi Forbidden chỉ là một cách tạm thời để bạn tiếp tục chạy mã mà không bị dừng lại hoàn toàn. Để truy cập vào các tài nguyên Reddit mà bạn bị từ chối, bạn cần kiểm tra lại quyền truy cập và xác thực tài khoản của mình.

In [19]:
from prawcore.exceptions import Forbidden

def prawGetData(limit, subredditName):
    topics = reddit.subreddit(subredditName).hot(limit=limit)
    commentInsert = []
    topicInsert = []
    topicNBR = 1
    for topic in topics:
        if (float(topicNBR)/limit)*100 in range(1, 100):
            print('•TOPIC:' + str(topic.id) + ' •COMPLETE: ' + str((float(topicNBR)/limit)*100) + ' % ****')
        topicNBR += 1
        try:
            topicInsert.append((topic.title, topic.selftext, topic.id, subredditName))
        except:
            pass
        try:
            for comment in topic.comments:
                try:
                    commentInsert.append((comment.body, comment.id, topic.title, topic.selftext, topic.id, subredditName))
                except Forbidden:
                    print("Forbidden: Unable to retrieve comment due to 403 Forbidden error.")
        except:
            pass
    print('––––––––––––––––––––––––––')
    print('INSERTING DATA INTO SQLITE')
    c.executemany('INSERT INTO topics VALUES (?,?,?,?)', topicInsert)
    print('INSERTED TOPICS')
    c.executemany('INSERT INTO comments VALUES (?,?,?,?,?,?)', commentInsert)
    print('INSERTED COMMENTS')
    conn.commit()


In [20]:
for subject in subreddits:
    prawGetData(limit=limit,subredditName=subject)

•TOPIC:147ecrr •COMPLETE: 1.0 % ****
•TOPIC:14atq9q •COMPLETE: 2.0 % ****
•TOPIC:14axqw4 •COMPLETE: 3.0 % ****
•TOPIC:14a8cx4 •COMPLETE: 4.0 % ****
•TOPIC:14azg2g •COMPLETE: 5.0 % ****
•TOPIC:14avaji •COMPLETE: 6.0 % ****
•TOPIC:14axuwt •COMPLETE: 8.0 % ****
•TOPIC:14axo38 •COMPLETE: 9.0 % ****
•TOPIC:14aoirf •COMPLETE: 10.0 % ****
•TOPIC:14alahr •COMPLETE: 11.0 % ****
•TOPIC:14b08vy •COMPLETE: 12.0 % ****
•TOPIC:14axovx •COMPLETE: 13.0 % ****
•TOPIC:14ax4pc •COMPLETE: 15.0 % ****
•TOPIC:14asmgk •COMPLETE: 16.0 % ****
•TOPIC:14aw8xm •COMPLETE: 17.0 % ****
•TOPIC:14avdku •COMPLETE: 18.0 % ****
•TOPIC:14auffa •COMPLETE: 19.0 % ****
•TOPIC:14auelc •COMPLETE: 20.0 % ****
•TOPIC:14alq83 •COMPLETE: 21.0 % ****
•TOPIC:14a388u •COMPLETE: 22.0 % ****
•TOPIC:14asv62 •COMPLETE: 23.0 % ****
•TOPIC:14aod9o •COMPLETE: 24.0 % ****
•TOPIC:14arq5r •COMPLETE: 25.0 % ****
•TOPIC:14arpnq •COMPLETE: 26.0 % ****
•TOPIC:14akwow •COMPLETE: 27.0 % ****
•TOPIC:14att4c •COMPLETE: 30.0 % ****
•TOPIC:14a4lf8 •COMP

Forbidden: received 403 HTTP response